# YOLO Image Retrieval

In [6]:
from ipyfilechooser import FileChooser
import os
import cv2
import numpy as np
import re
import random

# Paths Selection
Once run, you can select files multiple times without rerun the script

### Select Image Path
Select Image Folder

In [7]:
image_path_selector = FileChooser(os.getenv("HOME"))
image_path_selector.show_only_dirs = True
display(image_path_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

### Select Label Path
Select Label Folder

In [8]:
label_path_selector = FileChooser(os.getenv("HOME"))
label_path_selector.show_only_dirs = True
display(label_path_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

### Select Label File
Select *label.txt* file

In [9]:
label_file_selector = FileChooser(os.getenv("HOME"))
label_file_selector.filter_pattern = '*.txt'
display(label_file_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

### Select Output Path
Select Output Folder

*Note: Image will be written in {output_path}/{image_path name} directory*

*Example: Output path = {/home/x/Output}, Image path = /home/x/{Image} => Image will be written in /home/x/Output/Image/xx.jpg*

In [10]:
output_path_selector = FileChooser(os.getenv("HOME"))
output_path_selector.show_only_dirs = True
display(output_path_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

# Run after selecting all paths

In [11]:
#Create a Label class representing each bbox
class Label:
    def __init__(self, row):
        self.x = float(row[1])
        self.y = float(row[2])
        self.w = float(row[3])
        self.h = float(row[4])
        self.label = int(row[0])
        
    def getWidth(self):
        return self.w * self.width
    
    def getHeight(self):
        return self.h * self.height
    
    def setImageSize(self, size):
        self.width = size[1]
        self.height = size[0]
        
    def botleft(self, offsetx = 0, offsety = 0):
        return (int((self.x - (self.w / 2)) * self.width) + offsetx, int((self.y + (self.h / 2)) * self.height) + offsety)
    
    def topleft(self, offsetx = 0, offsety = 0):
        return (int((self.x - (self.w / 2)) * self.width) + offsetx, int((self.y - (self.h / 2)) * self.height) + offsety)
    
    def botright(self, offsetx = 0, offsety = 0):
        return (int((self.x + (self.w / 2)) * self.width) + offsetx, int((self.y + (self.h / 2)) * self.height) + offsety)
    

        
#Using dictionary to save all bounding boxes and labels for respective images
label_dict = dict()
label_dict.clear()

#define label path
label_path = label_path_selector.selected

#load label dict
if not os.path.isdir(label_path):
    
    print("not a directory")
    
else:
    
    for filename in os.listdir(label_path):
        
        if filename[-4:] != (".txt"):
            continue
        
        file = os.path.join(label_path, filename)
        
        with open(file, mode = 'r', newline = '') as f:
            
            label_list = list()
            update = True
            
            for row in f.read().splitlines():
                
                element = row[:-1].split(' ')
                
                if len(element) != 5:
                    update = False
                    break
        
                label_list.append(Label(element))
            
            if update:
                label_dict[filename[:-4]] = label_list
                
#Building class list
class_file = label_file_selector.selected
class_list = list()

with open(class_file, mode = 'r', newline = '') as f:
    class_list = f.read().splitlines()
    
    
#Building color list    
color_list = list()
#choose color for class
def random_color(lst):
    rgbl=[255,0,0]
    random.shuffle(rgbl)
    if (tuple(rgbl) in lst):
        return random_color(lst)
    return tuple(rgbl)

for i in range(len(class_list)):
    color_list.append(random_color(color_list))
    

##Write to Output
#check if image path is a directory
image_path = image_path_selector.selected
image_folder = image_path.split(os.path.sep)[-2]
output_path = os.path.join(output_path_selector.selected,image_folder)

if not os.path.isdir(image_path):
    
    print("not a directory")
else:
    #Create Output Path
    if  not os.path.exists(output_path):
        os.mkdir(output_path)
    
    for filename in os.listdir(image_path):
        
        #Only load Images in the folder
        splitter = filename.rfind(".")
        
        ext = filename[splitter:]
        
        isImage = re.search('(.)(jpg|png|jpeg|JPG|JPEG|PNG|bmp)', ext)
        
        if not isImage:
            continue
        
        mat = cv2.imread(os.path.join(image_path,filename))
        
        fix_h = 400

        ratio = mat.shape[1]/mat.shape[0]

        fix_w = int(fix_h * ratio)

        mat = cv2.resize(mat, (fix_w, fix_h), interpolation = cv2.INTER_AREA)

        #Duplicate image for easy checking
        mat2 = cv2.hconcat([mat, mat])
            
        name = filename[:splitter]
        
        if name in label_dict:

            for label in label_dict[name]:
                
                label.setImageSize(mat.shape)
                
                color = color_list[label.label]
                classes = class_list[label.label]

                #Rectangle size 
                rect_width = label.getWidth()
                rect_height = label.getHeight()
                min_dim = min(rect_width, rect_height)
                rect_thick = int(min_dim // 150)
                
                #Draw rectangle
                cv2.rectangle(mat2, label.topleft(), label.botright(), color, rect_thick)
                cv2.rectangle(mat2, label.topleft(offsetx = mat.shape[1]), label.botright(offsetx = mat.shape[1]), color, rect_thick)

                #Text box
                font = cv2.FONT_HERSHEY_COMPLEX_SMALL
                font_color = (255, 255, 255)
                thick = 10
                font_size = max(int(min_dim // 150),3)
                cv2.rectangle(mat2, label.botleft(offsety = -font_size * 15), label.botleft(offsetx = len(classes) * font_size * 15), color, -1)
                cv2.putText(mat2, classes, label.botleft(), font, font_size, font_color, font//2 , cv2.LINE_AA)
                
        cv2.imwrite(os.path.join(output_path,filename), mat2)